#MODELO: XGBOOST (EXTREME GRADIENT BOOSTING)

##1. INSTALACIÓN Y LIBRERÍAS

In [1]:


!pip install xgboost -q

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import gc
import warnings

warnings.filterwarnings('ignore')

##2. CARGA DE DATOS

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

!unzip udea*.zip > /dev/null
!wc *.csv

print("Cargando datasets...")
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.04GB/s]
   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total
Cargando datasets...


##3. PREPROCESAMIENTO Y FEATURE ENGINEERING

In [3]:
def procesar_datos_xgb(df_raw, es_train=True):
    """
    Limpieza y transformación optimizada para XGBoost.
    """
    df = df_raw.copy()

    # Eliminación de unarios
    for col in df.columns:
        if df[col].nunique(dropna=False) <= 1:
            df.drop(columns=[col], inplace=True)

    # Mapeos Manuales (Ordinales)
    mapa_si_no = {'Si': 1, 'No': 0, 'S': 1, 'N': 0}

    mapa_edu = {
        'Ninguno': 0, 'No sabe': 0, 'Primaria incompleta': 1, 'Primaria completa': 2,
        'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
        'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
        'Postgrado': 7
    }

    # Aplicar mapa binario a columnas que parecen de Si/No
    cols_binarias = [c for c in df.columns if 'TIENE' in c or 'PRIVADO' in c or 'PAGO' in c]
    for col in cols_binarias:
        df[col] = df[col].map(mapa_si_no)

    # Aplicar mapa educación
    for col in ['F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']:
        if col in df.columns:
            df[col] = df[col].map(mapa_edu)

    # C. Feature Engineering (Creación de variables)
    if 'F_EDUCACIONPADRE' in df.columns and 'F_EDUCACIONMADRE' in df.columns:
        df['CAPITAL_EDUCATIVO'] = df['F_EDUCACIONPADRE'].fillna(0) + df['F_EDUCACIONMADRE'].fillna(0)

    # D. Limpieza de Estrato
    if 'F_ESTRATOVIVIENDA' in df.columns:
        df['F_ESTRATOVIVIENDA'] = (df['F_ESTRATOVIVIENDA'].astype(str)
                                   .str.replace('Estrato ', '', regex=False)
                                   .str.replace('Sin Estrato', '0', regex=False)
                                   .replace('nan', '-1')) # XGBoost maneja -1 bien si es consistente
        df['F_ESTRATOVIVIENDA'] = pd.to_numeric(df['F_ESTRATOVIVIENDA'], errors='coerce').fillna(-1)

    # E. Conversión Final de Categóricas a Numéricas
    cat_cols = df.select_dtypes(include=['object']).columns

    for col in cat_cols:
        if col not in ['ID', 'RENDIMIENTO_GLOBAL']:
            # Convertimos a categoría y luego extraemos el código numérico
            # Los nulos quedan como -1
            df[col] = df[col].astype('category').cat.codes

    return df

# Ejecutar preprocesamiento
print("Procesando datos...")
X = procesar_datos_xgb(df_train.drop(columns=['RENDIMIENTO_GLOBAL', 'ID']))
X_submission = procesar_datos_xgb(df_test.drop(columns=['ID']))
y_raw = df_train['RENDIMIENTO_GLOBAL']
ids_test = df_test['ID']

# Codificar el Target (Obligatorio para XGBoost)
le = LabelEncoder()
y = le.fit_transform(y_raw)

# Limpieza memoria
del df_train, df_test
gc.collect()

print(f"Datos listos. Dimensiones: {X.shape}")

Procesando datos...
Datos listos. Dimensiones: (692500, 20)


## 4. CONFIGURACIÓN Y VALIDACIÓN DEL MODELO

In [4]:
# Split 80/20 estratificado
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definir el modelo XGBoost
# 'hist' método más rápido y eficiente para tablas grandes
clf_xgb = xgb.XGBClassifier(
    objective='multi:softmax',  # Clasificación multiclase
    num_class=len(le.classes_),
    n_estimators=1000,          # Máximo de árboles
    learning_rate=0.05,         # Velocidad moderada para buena precisión
    max_depth=9,                # Profundidad para capturar patrones complejos
    colsample_bytree=0.8,       # Evitar overfitting seleccionando columnas al azar
    subsample=0.8,              # Evitar overfitting seleccionando filas al azar
    tree_method='hist',         # Modo optimizado (rápido)
    random_state=42,
    n_jobs=-1,
    early_stopping_rounds=50    # Detener si no mejora en 50 rondas
)

print("\n--- Iniciando entrenamiento (Validación) ---")
clf_xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=100  # Mostrar progreso cada 100 iteraciones
)

# Evaluar
preds_val = clf_xgb.predict(X_val)
acc = accuracy_score(y_val, preds_val)
print(f"\n>>> Accuracy en Validación: {acc:.5f}")


--- Iniciando entrenamiento (Validación) ---
[0]	validation_0-mlogloss:1.37813
[100]	validation_0-mlogloss:1.21690
[200]	validation_0-mlogloss:1.20392
[300]	validation_0-mlogloss:1.20023
[400]	validation_0-mlogloss:1.19928
[500]	validation_0-mlogloss:1.19920
[504]	validation_0-mlogloss:1.19922

>>> Accuracy en Validación: 0.43458


## 5. ENTRENAMIENTO FINAL Y SUBMISSION

In [5]:
# Recuperar el mejor número de iteraciones
iteraciones_optimas = clf_xgb.best_iteration
# Agregamos un pequeño margen (10%) para aprovechar los datos extra de validación
iteraciones_finales = int(iteraciones_optimas * 1.1)

print(f"\n--- Re-entrenando con TODO el dataset ({iteraciones_finales} rondas) ---")

modelo_final = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=len(le.classes_),
    n_estimators=iteraciones_finales, # Usamos el número óptimo
    learning_rate=0.05,
    max_depth=9,
    colsample_bytree=0.8,
    subsample=0.8,
    tree_method='hist',
    random_state=42,
    n_jobs=-1
)

modelo_final.fit(X, y)

print("Generando predicciones para el archivo final...")
preds_test_codigos = modelo_final.predict(X_submission)
preds_test_labels = le.inverse_transform(preds_test_codigos)

# Crear DataFrame y Guardar
nombre_archivo = 'submission_xgboost.csv'
df_sub = pd.DataFrame({'ID': ids_test, 'RENDIMIENTO_GLOBAL': preds_test_labels})
df_sub.to_csv(nombre_archivo, index=False)

print(f"¡Éxito! Archivo guardado como: {nombre_archivo}")


--- Re-entrenando con TODO el dataset (499 rondas) ---
Generando predicciones para el archivo final...
¡Éxito! Archivo guardado como: submission_xgboost.csv
